<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Unix---awk-and-Makefiles" data-toc-modified-id="Introduction-to-Unix---awk-and-Makefiles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Unix - awk and Makefiles</a></span><ul class="toc-item"><li><span><a href="#Summary-of-the-course-today" data-toc-modified-id="Summary-of-the-course-today-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Summary of the course today</a></span></li></ul></li><li><span><a href="#Working-with-tabular-files:-Awk" data-toc-modified-id="Working-with-tabular-files:-Awk-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Working with tabular files: Awk</a></span><ul class="toc-item"><li><span><a href="#Pros-of-awk-vs-Excel:" data-toc-modified-id="Pros-of-awk-vs-Excel:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Pros of awk vs Excel:</a></span></li><li><span><a href="#Cons-of-awk:" data-toc-modified-id="Cons-of-awk:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Cons of awk:</a></span></li><li><span><a href="#the-GFF3-format-for-Genomic-Annotations" data-toc-modified-id="the-GFF3-format-for-Genomic-Annotations-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>the GFF3 format for Genomic Annotations</a></span></li><li><span><a href="#Basic-AWK-syntax:-filters" data-toc-modified-id="Basic-AWK-syntax:-filters-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Basic AWK syntax: filters</a></span></li><li><span><a href="#Awk---the-ACTION-component" data-toc-modified-id="Awk---the-ACTION-component-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Awk - the ACTION component</a></span><ul class="toc-item"><li><span><a href="#Operations-between-columns" data-toc-modified-id="Operations-between-columns-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>Operations between columns</a></span></li></ul></li><li><span><a href="#Awk---the-PATTERN-component" data-toc-modified-id="Awk---the-PATTERN-component-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Awk - the PATTERN component</a></span><ul class="toc-item"><li><span><a href="#Filtering-by-a-string" data-toc-modified-id="Filtering-by-a-string-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Filtering by a string</a></span></li><li><span><a href="#Exercise:-How-many-chromosomes-there-are-in-the-file?" data-toc-modified-id="Exercise:-How-many-chromosomes-there-are-in-the-file?-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Exercise: How many chromosomes there are in the file?</a></span></li><li><span><a href="#Filtering-by-numerical-conditions" data-toc-modified-id="Filtering-by-numerical-conditions-2.6.3"><span class="toc-item-num">2.6.3&nbsp;&nbsp;</span>Filtering by numerical conditions</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.6.4"><span class="toc-item-num">2.6.4&nbsp;&nbsp;</span>Exercise</a></span></li><li><span><a href="#Exercise!" data-toc-modified-id="Exercise!-2.6.5"><span class="toc-item-num">2.6.5&nbsp;&nbsp;</span>Exercise!</a></span></li></ul></li><li><span><a href="#Splitting-a-file-in-multiple-files,-according-to-value-of-a-column" data-toc-modified-id="Splitting-a-file-in-multiple-files,-according-to-value-of-a-column-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Splitting a file in multiple files, according to value of a column</a></span><ul class="toc-item"><li><span><a href="#Extracting-gene-symbols-from-a-string-in-awk" data-toc-modified-id="Extracting-gene-symbols-from-a-string-in-awk-2.7.1"><span class="toc-item-num">2.7.1&nbsp;&nbsp;</span>Extracting gene symbols from a string in awk</a></span></li><li><span><a href="#How-to-print-both-gene-coordinates-and-gene-symbols?" data-toc-modified-id="How-to-print-both-gene-coordinates-and-gene-symbols?-2.7.2"><span class="toc-item-num">2.7.2&nbsp;&nbsp;</span>How to print both gene coordinates and gene symbols?</a></span></li></ul></li></ul></li><li><span><a href="#Makefiles" data-toc-modified-id="Makefiles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Makefiles</a></span><ul class="toc-item"><li><span><a href="#Defining-pipelines-with-Makefiles" data-toc-modified-id="Defining-pipelines-with-Makefiles-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Defining pipelines with Makefiles</a></span></li><li><span><a href="#How-to-run-Makefile-rules" data-toc-modified-id="How-to-run-Makefile-rules-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>How to run Makefile rules</a></span><ul class="toc-item"><li><span><a href="#Exercise!" data-toc-modified-id="Exercise!-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Exercise!</a></span></li></ul></li></ul></li><li><span><a href="#What-have-you-learned-today?" data-toc-modified-id="What-have-you-learned-today?-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>What have you learned today?</a></span><ul class="toc-item"><li><span><a href="#Dinner-time!!" data-toc-modified-id="Dinner-time!!-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Dinner time!!</a></span></li></ul></li></ul></div>

In [21]:
# Configuration - please ignore - this should not appear in the slideshow
alias grep='grep --color'
cd
cd workspace/peb_unix_intro/exercises/
# Please do not run this - it is only to format the slides
# if you did, type "unalias less"
alias less="head"  

# Introduction to Unix - awk and Makefiles

**<font color='darkgreen'> Welcome to the Programming for Evolutionary Biology workshop!! </font>** 


Giovanni M. Dall'Olio. Data Strategy and Design, GSK. March 2020. 

Quick link to slides online: https://tinyurl.com/evop-unix



All materials available here: https://github.com/dalloliogm/peb_unix_intro/archive/master.zip

Press space or down key to continue.

## Summary of the course today

- Morning first half: Basic Terminal commands, First Login to Linux
- Morning second half: Login to a Remote Unix server, browsing file contents
- Afternoon first half: Finding patterns in a file with grep; piping commands; cut, sort and uniq
- <font color="blue">Afternoon second half</font>: awk and sed; makefiles

# Working with tabular files: Awk


The `awk` command is the equivalent of Excel/Calc for the command line. 

It allows to filter files by specific columns, execute numerical operations, or change to the order of the columns.



## Pros of awk vs Excel:
- Very low memory requirements
- Only one line at a time is processed, so `awk` can parse very large files without using resources

## Cons of awk:
- Need to define operations before opening the file
- large I/O usage
- only works on Text files

## the GFF3 format for Genomic Annotations

The exercises on `awk` will be based on GFF3, which is another format to store gene annotations. It's similar to BED, but the column order is different.

The file `genes/chr8.gff` contains an example:

In [5]:
head genes/chr8.gff

##gff-version 3
##source-version rtracklayer 1.34.1
##date 2017-02-27
##genome-build .	hg19
chr8	rtracklayer	sequence_feature	18248755	18258723	.	+	.	gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8	rtracklayer	sequence_feature	100549014	100549089	.	-	.	gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8	rtracklayer	sequence_feature	144895127	144895212	.	-	.	gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8	rtracklayer	sequence_feature	145619364	145619445	.	-	.	gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8	rtracklayer	sequence_feature	91970706	91997485	.	-	.	gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8	rtracklayer	sequence_feature	74332309	74353753	.	+	.	gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126


As you can see it is a tab-separated file.

The **GFF** (General Feature Format) format specifications are defined [here](https://genome.ucsc.edu/FAQ/FAQformat.html#format3), but in short:

- `col1`, `col4` and `col5` contain the chromosome name and genomic coordinates (start and end),
- `col2` describes the tool or resource that generated the annotation,
- `col3` describe the type of feature (e.g. gene, transcript, exon, TF binding site, Histone Acetylation mark, etc...
- `col9` column contains several fields, separated by a semicolon


## Basic AWK syntax: filters

The basic AWK syntax is the following:


    awk 'PATTERN {ACTION}' filename


- **PATTERN**: select which rows to print
        e.g. print only rows containing a specific word or pattern
        
- **ACTION**: what to do on each line (e.g. print a specific column, sum them, etc..)
        e.g. print only column #1 and #2

Here is an example awk command:

In [9]:
# This prints all the lines on chromosome 8 and with chromosomal position between 10000 and 20000
# We'll go through this code in a minute
# 
awk '$1=="chr8" && $4>100000 && $5<200000 {print $0}' genes/chr8.gff

chr8	rtracklayer	sequence_feature	182200	197339	.	+	.	gene_id=169270;symbol=ZNF596;exerc1=FALSE;ID=169270
chr8	rtracklayer	sequence_feature	116086	117024	.	-	.	gene_id=441308;symbol=OR4F21;exerc1=FALSE;ID=441308
chr8	rtracklayer	sequence_feature	158345	182318	.	-	.	gene_id=644128;symbol=RPL23AP53;exerc1=FALSE;ID=644128


## Awk - the ACTION component

    awk 'PATTERN {ACTION}' filename

The **ACTION** component of an awk command describes what to do with each line.

- Actions must be included in curly brackets { }
- The first column is `$1`; the second `$2`; and so on. `$0` to print everything
- The whole `awk` statement must be included in quotes (pref single quotes)


In [51]:
# Basic awk command - similar to using cut -f 1,4,5,9
awk '{print $1, $4, $5, $9}' genes/chr8.gff | head
# Try comparing with:
# cut -f 1,4,5,9  genes/chr8.gff  | head

##gff-version   
##source-version   
##date   
##genome-build   
chr8 18248755 18258723 gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8 100549014 100549089 gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8 144895127 144895212 gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8 145619364 145619445 gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8 91970706 91997485 gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8 74332309 74353753 gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126


In [54]:
# To skip the headers, use grep with the -v option 
# grep -v '#' genes/chr8.gff |  awk '{print $1, $4, $5, $9}'  |  head

###  Operations between columns

`awk` allows to do mathematical operations between columns.

For example we can calculate the gene length, from gene start/end.

Note: GFF files are 1-based, so we need to add 1 to the formula

In [22]:
# Calculating gene length
awk '{print $1, $4, $5, $5-$4+1, $9}' genes/chr8.gff  |  head

##gff-version   1 
##source-version   1 
##date   1 
##genome-build   1 
chr8 18248755 18258723 9969 gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8 100549014 100549089 76 gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8 144895127 144895212 86 gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8 145619364 145619445 82 gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8 91970706 91997485 26780 gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8 74332309 74353753 21445 gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126


## Awk - the PATTERN component

    awk 'PATTERN {ACTION}' filename

The PATTERN component of an awk command specifies filtering conditions.

- Filter rows where a specific column matches a string, or values are higher than a cut-off
- Columns can be referred as `$1`, `$2`, etc
- The whole awk statement must be included in quotes

### Filtering by a string

The `awk` statement below will filter all the lines where the first column is `chr8`.

Note the syntax to match a specific string: `$COLUMN_NUMBER ~ /PATTERN/`

In [23]:
awk '$1 ~ /chr8/ ' genes/chr8.gff | head


chr8	rtracklayer	sequence_feature	18248755	18258723	.	+	.	gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8	rtracklayer	sequence_feature	100549014	100549089	.	-	.	gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8	rtracklayer	sequence_feature	144895127	144895212	.	-	.	gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8	rtracklayer	sequence_feature	145619364	145619445	.	-	.	gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8	rtracklayer	sequence_feature	91970706	91997485	.	-	.	gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8	rtracklayer	sequence_feature	74332309	74353753	.	+	.	gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126
chr8	rtracklayer	sequence_feature	144816310	144828507	.	+	.	gene_id=100128338;symbol=FAM83H-AS1;exerc1=FALSE;ID=100128338
chr8	rtracklayer	sequence_feature	144077245	144079080	.	+	.	gene_id=100128627;symbol=CDC42P3;exerc1=FALSE;ID=100128627
chr8	rtracklayer	sequence_feature	30239635	30242917	.	-	.	gene_id=100

### Exercise: How many chromosomes there are in the file?

- Use `awk` to print all the lines that match the string `chr`
- Use `wc` to count how many lines there are (see `man wc`)
- Bonus: Use `cut`, `sort` and `uniq` to find how many unique chromosomes are present

In [33]:
awk '$1 ~ /chr/ ' genes/chr8.gff | wc

    818    7364   88987


In [35]:
awk '$1 ~ /chr/' genes/chr8.gff | cut -f 1 | sort | uniq -c

    818 chr8


### Filtering by numerical conditions

`awk` can also filter by numerical conditions.

We can also specify multiple conditions, using `&&` (AND) and `||` (OR).

The following will filter genes on chr8, from 5,000,000 to 10,000,000:

In [55]:
awk '$1 ~ /chr8/ && $4 > 5000000 && $5 < 10000000 ' genes/chr8.gff | head


chr8	rtracklayer	sequence_feature	7143733	7212876	.	-	.	gene_id=100128890;symbol=FAM66B;exerc1=TRUE;ID=100128890
chr8	rtracklayer	sequence_feature	7215498	7220490	.	-	.	gene_id=100131980;symbol=ZNF705G;exerc1=TRUE;ID=100131980
chr8	rtracklayer	sequence_feature	7812535	7866277	.	+	.	gene_id=100132103;symbol=FAM66E;exerc1=TRUE;ID=100132103
chr8	rtracklayer	sequence_feature	7783859	7809935	.	+       / Cows in \
chr8	rtracklayer	sequence_feature	6261077	6264069	.	        | the      |
chr8	rtracklayer	sequence_feature	7272385	7274354	.	-       \ Genome!  /
chr8	rtracklayer	sequence_feature	7946463	7946611	.	.	  ---------
chr8	rtracklayer	sequence_feature	6602685	6602765	.	+	 ||  ^__^
chr8	rtracklayer	sequence_feature	8905955	8906028	.	+	 ||  (oo)\\_______
chr8	rtracklayer	sequence_feature	6602689	6602761	.	-	     (__)\\       )\\/\


### Exercise

Print all the lines between chromosomal positions 5,000,000 and 10,000,000 (columns 4 and 5)

- Use the syntax ```awk 'PATTERN' filename```
- PATTERN is a condition to filter column \\$4 > 5000000 and column \\$5 < 10000000
- filename is genes/chr8.gff

In [71]:
awk '$4 > 5000000 && $5 < 10000000 ' genes/chr8.gff | wc 


      47     425    4709


The first number is the number of rows; the other numbers are the number of words, and the number of characters in total.


### Exercise!

Calculate the lenght of the gene POU5F1B.

Find the Gene whose gene_id is equal to that number.

- Find all the lines of genes/chr8.gff where column \$9 ~ \/POU5F1B\/

- Print the length of the gene, by subtracting column \\$4 from column \$5. This will give you a number.

- Find the line of genes/chr8.gff where column $9 contains such number

In [12]:
awk '$9 ~ /POU5F1B/ {print $5-$4}' genes/chr8.gff 


1584


In [8]:
awk '$9 ~ /gene_id=1584/ {print $0}' genes/chr8.gff 

chr8	Well_done	Great_job	143953773	143961236	.	-	.	gene_id=1584;symbol=CYP11B1;exerc1=FALSE;ID=1584


## Splitting a file in multiple files, according to value of a column

The example.bed file contains lines belonging to different chromosomes (column 1). 

We can split these in separate files, using the following syntax:

In [75]:
#awk '{print>$1".txt"}' example.bed # doesn't work on MacBooc
awk '{print>$1}' genes/example.bed

In [76]:
ls

1_browsing_textfiles.txt	chr8
2_searching_patterns.txt	genes
bos_taurus.txt			multiplefiles
chr20				old_files


### Extracting gene symbols from a string in awk

Notice how in in this file the gene symbols are embedded in a long string containing other information:

    gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10


Is there a way to extract just the gene ID and Symbols from this text, while also keeping the gene coordinates?


Extracting the gene symbols in this case is not trivial, but this example applies to several formats in bioinformatics, so it is useful to see how to do it.

The first step is to use the -F operator in awk to specify a different fields separator - in this case, the semicolon:



In [54]:
grep -v '^#' genes/chr8.gff | awk -F';' '{print $2}' | head

symbol=NAT2
symbol=MIR875
symbol=MIR937
symbol=MIR939
symbol=C8orf88
symbol=STAU2-AS1
symbol=FAM83H-AS1
symbol=CDC42P3
symbol=RBPMS-AS1
symbol=FAM66B


### How to print both gene coordinates and gene symbols?

We have to use two consecutive awk searches. 

- in the first awk statement, we will extract the chromosome, start, end columns, as well as the string containing the symbol string (column 1, 4,5, and 9);
- in the second statement, we will split the symbol string to extract the symbol id  

In [58]:
grep -v '^#' genes/chr8.gff | awk '{print $1, $4, $5, $9}' | grep -v '^#' | awk -F';' '{print $1, $2}' | head

chr8 18248755 18258723 gene_id=10 symbol=NAT2
chr8 100549014 100549089 gene_id=100126309 symbol=MIR875
chr8 144895127 144895212 gene_id=100126338 symbol=MIR937
chr8 145619364 145619445 gene_id=100126351 symbol=MIR939
chr8 91970706 91997485 gene_id=100127983 symbol=C8orf88
chr8 74332309 74353753 gene_id=100128126 symbol=STAU2-AS1
chr8 144816310 144828507 gene_id=100128338 symbol=FAM83H-AS1
chr8 144077245 144079080 gene_id=100128627 symbol=CDC42P3
chr8 30239635 30242917 gene_id=100128750 symbol=RBPMS-AS1
chr8 7143733 7212876 gene_id=100128890 symbol=FAM66B


# Makefiles


You may have noticed that the unix_intro folder contains a file called Makefile. 
 
What is a Makefile? Let's have a look at its contents first: 

In [14]:
cd ..
head Makefile

# This is a Makefile, which will be explained later in the course.
# Please don't look at it yet :-)

publish: slides_bash commit
	echo "convert the slides to pdf, commit, and push to github"
	git push


test_exercises: start help ignorecase multiplefiles
generate_exercises: generate_grep generate_awk


Press space or the down key to continue

## Defining pipelines with Makefiles

Makefiles are a basic way to define pipelines of shell commands.

Nowadays there are more sophisticated tools available, but most of these are based on Makefiles.


A Makefile is a collection of "rules".

Each of these rules follows this basic syntax is:

```
target: prerequisites
    commands to execute
```

As you can see in the Makefile included, most of the rules allow to regenerate the exercise files, or to execute some commands without having to type them everytime.

For example, the rule "testrule" is associated to two echo commands.

## How to run Makefile rules

To execute a rule in the Makefile, simply type:

```
make [name of the rule]
```

For example:



In [25]:
make testrule

echo this is a Makefile rule
this is a Makefile rule
echo You can associate it to as many commands you want
You can associate it to as many commands you want


The program "make" will automatically detect any file named "Makefile" in the current directory, and execute any rule with the specific name.

Rules can also be nested together. For example the two rules "test_exercises" and "generate_exercises" at the beginning of the file are a way to call several other rules together.

### Exercise!

Explore the Makefile in the exercises folder.

Run the rule for the awk1 exercise, and check if it is correct.

- cd into the main folder for these exercises (peb_unix_intro)
- Open the Makefile with less, to see the contents
- Return to the terminal, and run the "awk1" rule
- To run a Makefile rule, type ```make RULE```

# What have you learned today?

- The Unix command line is fun! (hopefully)
- There is an incredible number of many Unix Commands to deal with text files
- Unix commands are specialized to do single task, and are meant to be combined together with the pipe operator

## Dinner time!!
